In [1]:
from IPython.display import Image

_unchecked_

# CNTK 599A: 序列与文本数据的序列网络

## 介绍和背景

本教程将带您完成序列到顺序网络的基础知识, 以及如何在 Microsoft 认知工具包中实现它们。特别是, 我们将实现一个序列到序列模型来执行字形的音素翻译。我们将从一些基本理论开始, 然后更详细地解释数据, 以及如何下载它。

Andrej 索腾卡帕锡对神经网络体系结构的五范例有一个[很好的可视化](http://karpathy.github.io/2015/05/21/rnn-effectiveness/):

In [2]:
# Figure 1
Image(url="http://cntk.ai/jup/paradigms.jpg", width=750)

_unchecked_

在本教程中, 我们将讨论第四范式: 多到多, 也称为序列到序列网络。输入是一个具有动态长度的序列, 输出也是一个具有一定动态长度的序列。这是多到一个范例的逻辑扩展, 因为以前我们预测某种类别 (这很容易成为 `V` 单词的其中之一 `V` 是整个词汇表), 现在我们要预测这些类别的整个序列。

顺序序列网络的应用几乎是无限的。这是一个自然适合机器翻译 (如英文输入序列, 法国输出序列);自动文本摘要 (如完整的文档输入序列, 摘要输出序列);词到发音模型 (例如字符 [字形] 输入序列, 发音 [音素] 输出序列);甚至解析树生成 (例如, 常规文本输入、平面解析树输出)。

## 基础理论

序列到序列模型由两个主要部分组成: (1) 编码器;和 (2) 解码器。编码器和解码器都是递归神经网络 (RNN) 层, 可以使用香草 RNN、LSTM 或 GRU 细胞来实现 (这里我们将使用 LSTM)。在基本序列到序列模型中, 编码器将输入序列处理成一个固定的表示形式, 并将其作为上下文送入解码器。解码器然后使用一些机制 (下面讨论) 解码被处理的信息入一个输出序列。解码器是一种语言模型, 它通过编码器增强一些 "强上下文", 因此它生成的每个符号都被反馈到解码器中以增加上下文 (如传统的 LM)。对于英语到德语的翻译任务, 最基本的设置可能类似于以下内容:

In [3]:
# Figure 2
Image(url="http://cntk.ai/jup/s2s.png", width=700)

_unchecked_

基本的顺序顺序网络将信息从编码器传递到解码器, 通过初始化解码器 RNN 以最终的隐藏状态作为其初始隐藏状态。输入然后是一个 "序列开始" 标记 ( `<s>` 在上面的图中), 它使解码器开始生成输出序列。然后, 它在该步骤中生成的任何单词 (或注释或图像等) 作为下一步的输入。解码器继续生成输出, 直到它命中特殊的 "结束序列" 标记 ( `</s>` 上面)。

一个更复杂和功能强大的基本顺序序列网络的版本使用注意模型。当上述设置工作正常时, 当输入序列长时, 它可以开始分解。在每个步骤中, 隐藏状态 `h` 将使用最新的信息进行更新, 因此 `h` 在处理每个令牌时, 可能会在信息中被 "稀释"。更进一步, 即使有一个相对较短的序列, 最后一个令牌总是会得到最后的发言权, 因此思想向量会有点偏向/加权的最后一个词。为了解决这个问题, 我们使用了一个 "注意" 机制, 使解码器不仅可以从输入的所有隐藏状态中进行查找, 而且还能了解每个解码步骤中的隐藏状态, 以便将最大的权重放在上面。我们将在本教程的稍后版本中讨论注意实现。

_unchecked_

## 问题: 字形-音素转换

[字形](https://en.wikipedia.org/wiki/Grapheme)到[音素](https://en.wikipedia.org/wiki/Phoneme)问题是一个翻译任务, 它将单词的字母作为输入序列 (分析是书写系统的最小单位), 并输出相应的音素;也就是组成一种语言的声音的单位。换句话说, 该系统的目的是产生一个 unambigious 的表示, 如何发音一个给定的输入字。

### 例如

分析或字母被翻译成相应的音素:

>**字形**: **|**T **|**一个**|**N **|**G **|**E **|**R **|**


**音素**: **|** ~~ **|** ~ AE **|** **None** ~ NG | 二**None** | 空**|**空**|**

_unchecked_

## 任务和模型结构

如上所述, 我们感兴趣的解决的任务是创建一个模型, 它以某一序列作为输入, 并根据输入的内容生成输出序列。模型的任务是学习从输入序列到它将生成的输出序列的映射。编码器的工作是拿出一个很好的表示的输入, 解码器可以用来产生一个良好的输出。对于编码器和解码器, LSTM 在这方面做得很好。

我们将使用 CNTK 块库中的 LSTM 实现。这实现了 LSTM 的 "智慧", 我们或多或少可以把它看成一个黑箱。然而, 重要的是要理解的是, 在实现 RNN 时, 有两件事需要考虑: 重复性, 即在序列上展开的网络, 和块, 这是网络的一部分, 为序列的每个元素运行。我们只需要实现重复。

它有助于将重复性看作是一个函数, 它一直调用 `step(x)` 块 (在我们的情况下, LSTM)。在高级别, 它看起来像这样:

"' 类 LSTM {浮动 hidden_state

    `init(initial_value):
        hidden_state = initial_value
    
    step(x):
        hidden_state = LSTM_function(x, hidden_state)
        return hidden_state
    `

}
```

因此, 对 `step(x)` 函数的每个调用都需要一些输入 `x` , 修改内部 `hidden_state` , 并返回它。因此, 在每个输入 `x` 中, `hidden_state` 的值都将演变。下面我们将导入一些必需的功能, 然后实现使用此机制的重复。

_unchecked_

## 导入 CNTK 和其他有用的库

CNTK 是一个 Python 模块, 它包含多个子, 如 `io` 、 `learner` 、 `graph` 等。我们也广泛使用 numpy。

In [4]:
from __future__ import print_function
import numpy as np
import os

import cntk as C

_unchecked_

在下面的块中, 我们检查是否在 CNTK 的内部测试机器中运行这个笔记本, 寻找在那里定义的环境变量。然后, 我们选择正确的目标设备 (GPU vs CPU) 来测试这个笔记本。在其他情况下, 我们使用 CNTK 的默认策略来使用最好的设备 (GPU, 如果可用的话, 其他 CPU)。

In [5]:
# Select the right target device when this notebook is being tested:
if 'TEST_DEVICE' in os.environ:
    if os.environ['TEST_DEVICE'] == 'cpu':
        C.device.try_set_default_device(C.device.cpu())
    else:
        C.device.try_set_default_device(C.device.gpu(0))

_unchecked_

<h2>下载数据</h2>
<p>在本教程中, 我们将使用从 http://www.speech.cs.cmu.edu/cgi-bin/cmudict CMUDict (版本 0.7b) 数据集的一个轻松的预处理版本。CMUDict 数据是卡耐基梅隆大学发音字典是一个开放源码机器可读的发音词典为北美洲英语。数据采用 CNTKTextFormatReader 格式。下面是来自数据的一个序列对, 其中输入序列 (S0) 在左列中, 输出序列 (S1) 在右边:</p>
<p><code>0   |S0 3:1 |# &lt;s&gt;  |S1 3:1 |# &lt;s&gt;
0   |S0 4:1 |# A    |S1 32:1 |# ~AH
0   |S0 5:1 |# B    |S1 36:1 |# ~B
0   |S0 4:1 |# A    |S1 31:1 |# ~AE
0   |S0 7:1 |# D    |S1 38:1 |# ~D
0   |S0 12:1 |# I   |S1 47:1 |# ~IY
0   |S0 1:1 |# &lt;/s&gt; |S1 1:1 |# &lt;/s&gt;</code></p>
<p>下面的代码将下载所需的文件 (培训、上面的单层验证和一个小的词汇文件) 并将它们放在一个本地文件夹中 (培训文件是 ~ 34 mb, 测试是 ~ 4 mb, 验证文件和词汇文件都少于 1KB)。</p>

In [6]:
import requests

def download(url, filename):
    """ utility function to download a file """
    response = requests.get(url, stream=True)
    with open(filename, "wb") as handle:
        for data in response.iter_content():
            handle.write(data)

data_dir = os.path.join('..', 'Examples', 'SequenceToSequence', 'CMUDict', 'Data')
# If above directory does not exist, just use current.
if not os.path.exists(data_dir):
    data_dir = '.'

valid_file = os.path.join(data_dir, 'tiny.ctf')
train_file = os.path.join(data_dir, 'cmudict-0.7b.train-dev-20-21.ctf')
vocab_file = os.path.join(data_dir, 'cmudict-0.7b.mapping')

files = [valid_file, train_file, vocab_file]

for file in files:
    if os.path.exists(file):
        print("Reusing locally cached: ", file)
    else:
        url = "https://github.com/Microsoft/CNTK/blob/release/2.4/Examples/SequenceToSequence/CMUDict/Data/%s?raw=true"%file
        print("Starting download:", file)
        download(url, file)
        print("Download completed")

Reusing locally cached:  ..\Examples\SequenceToSequence\CMUDict\Data\tiny.ctf
Reusing locally cached:  ..\Examples\SequenceToSequence\CMUDict\Data\cmudict-0.7b.train-dev-20-21.ctf
Reusing locally cached:  ..\Examples\SequenceToSequence\CMUDict\Data\cmudict-0.7b.mapping


_unchecked_

### 选择笔记本运行模式

有两种运行模式:-*快速模式*: `isFast` 设置为 `True` 。这是笔记本的默认模式, 这意味着我们在有限的数据上进行较少的迭代或培训/测试。这确保了笔记本的功能正确性, 虽然所生产的模型远非完成的培训所能产生的。

- *慢速模式*: 我们建议用户将此标志设置为 `False` 一旦用户已经熟悉了笔记本内容, 并希望从使用不同的参数进行培训的更长时间内了解如何运行笔记本。

In [7]:
isFast = True

_unchecked_

## 读者

为了有效地收集数据, 随机进行培训, 并将其传递给网络, 我们使用 CNTKTextFormat 阅读器。我们将创建一个小函数, 它将在定义数据流名称的培训 (或测试) 时调用, 以及如何在原始培训数据中引用它们。

In [8]:
# Helper function to load the model vocabulary file
def get_vocab(path):
    # get the vocab for printing output sequences in plaintext
    vocab = [w.strip() for w in open(path).readlines()]
    i2w = { i:ch for i,ch in enumerate(vocab) }

    return (vocab, i2w)

# Read vocabulary data and generate their corresponding indices
vocab, i2w = get_vocab(vocab_file)

input_vocab_size = len(vocab)
label_vocab_size = len(vocab)

In [9]:
# Print vocab and the correspoding mapping to the phonemes
print("Vocabulary size is", len(vocab))
print("First 15 letters are:")
print(vocab[:15])
print()
print("Print dictionary with the vocabulary mapping:")
print(i2w)

Vocabulary size is 69
First 15 letters are:
["'", '</s>', '<s/>', '<s>', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']

Print dictionary with the vocabulary mapping:
{0: "'", 1: '</s>', 2: '<s/>', 3: '<s>', 4: 'A', 5: 'B', 6: 'C', 7: 'D', 8: 'E', 9: 'F', 10: 'G', 11: 'H', 12: 'I', 13: 'J', 14: 'K', 15: 'L', 16: 'M', 17: 'N', 18: 'O', 19: 'P', 20: 'Q', 21: 'R', 22: 'S', 23: 'T', 24: 'U', 25: 'V', 26: 'W', 27: 'X', 28: 'Y', 29: 'Z', 30: '~AA', 31: '~AE', 32: '~AH', 33: '~AO', 34: '~AW', 35: '~AY', 36: '~B', 37: '~CH', 38: '~D', 39: '~DH', 40: '~EH', 41: '~ER', 42: '~EY', 43: '~F', 44: '~G', 45: '~HH', 46: '~IH', 47: '~IY', 48: '~JH', 49: '~K', 50: '~L', 51: '~M', 52: '~N', 53: '~NG', 54: '~OW', 55: '~OY', 56: '~P', 57: '~R', 58: '~S', 59: '~SH', 60: '~T', 61: '~TH', 62: '~UH', 63: '~UW', 64: '~V', 65: '~W', 66: '~Y', 67: '~Z', 68: '~ZH'}


_unchecked_

我们将使用上面的信息为我们的培训数据创建一个阅读器。让我们现在创建它:

In [10]:
def create_reader(path, randomize, size=C.io.INFINITELY_REPEAT):
    return C.io.MinibatchSource(C.io.CTFDeserializer(path, C.io.StreamDefs(
        features  = C.io.StreamDef(field='S0', shape=input_vocab_size, is_sparse=True),
        labels    = C.io.StreamDef(field='S1', shape=label_vocab_size, is_sparse=True)
    )), randomize=randomize, max_samples = size)

# Train data reader
train_reader = create_reader(train_file, True)

# Validation/Test data reader
valid_reader = create_reader(valid_file, False)

_unchecked_

### 现在, 让我们设置我们的模型参数..。

_unchecked_

我们的输入词汇量是 69, 而那些也代表了标签。此外, 我们有1隐藏层与128节点。

In [11]:
model_dir = "." # we downloaded our data to the local directory above # TODO check me

# model dimensions
input_vocab_dim  = input_vocab_size
label_vocab_dim  = label_vocab_size
hidden_dim = 128
num_layers = 1

_unchecked_

## 步骤 1: 将输入设置到网络

### CNTK 中的动态轴 (关键概念)

理解 CNTK 的一个重要概念是两种类型的轴的想法:-**静态轴**, 它是变量形状的传统轴, 而-**动态轴**, 它具有未知的维度, 直到变量绑定到实数计算时的数据。

动态轴在回归神经网络的世界中尤为重要。而不是必须提前决定一个最大的序列长度, 填充你的序列到那个大小, 浪费计算, CNTK 的动态轴允许可变序列长度, 自动包装在 minibatches 尽可能有效。

在设置序列时, 有一个重要的*两个动态轴*需要考虑。第一个是*批处理轴*, 该轴是多个序列的分批排列的坐标轴。第二个是特定于该序列的动态轴。后者是特定于特定输入的, 因为数据中的序列长度可变。例如, 在序列网络中, 我们有两个序列:**输入序列**和**输出 (或 "标签") 序列**。使这种类型的网络如此强大的一个因素是输入序列和输出序列的长度不必相互对应。因此, 输入序列和输出序列都需要它们自己唯一的动态轴。

在定义输入到网络时, 我们设置所需的动态轴和输入变量的形状。下面, 我们定义输入的形状 (词汇大小), 创建它们的动态轴, 最后创建表示网络中输入节点的输入变量。

In [12]:
# Source and target inputs to the model
input_seq_axis = C.Axis('inputAxis')
label_seq_axis = C.Axis('labelAxis')

raw_input = C.sequence.input_variable(shape=(input_vocab_dim), sequence_axis=input_seq_axis, name='raw_input')

raw_labels = C.sequence.input_variable(shape=(label_vocab_dim), sequence_axis=label_seq_axis, name='raw_labels')

_unchecked_

### 问题

1. 为什么输入变量的形状与我们的字典的大小对应于顺序网络？

_unchecked_

## 步骤 2: 定义网络

如前所述, 序列-顺序网络是, 在其最基本的, 一个 RNN 编码器后跟一个 RNN 译码器, 和一个稠密的输出层。我们可以在几行中使用层库来完成这项工作, 但是让我们在不增加太多复杂性的情况下更详细地进行一些事情。第一步是对输入数据执行一些操作;让我们看看下面的代码, 然后讨论我们在做什么。

In [13]:
# Instantiate the sequence to sequence translation model
input_sequence = raw_input

# Drop the sentence start token from the label, for decoder training
label_sequence = C.sequence.slice(raw_labels,
                       1, 0, name='label_sequence') # <s> A B C </s> --> A B C </s>
label_sentence_start = C.sequence.first(raw_labels)   # <s>

is_first_label = C.sequence.is_first(label_sequence)  # 1 0 0 0 ...
label_sentence_start_scattered = C.sequence.scatter(  # <s> 0 0 0 ... (up to the length of label_sequence)
    label_sentence_start, is_first_label)

_unchecked_

我们有两个输入变量, `raw_input` and `raw_labels` 。通常, 标签不必是网络定义的一部分, 因为当我们将网络的输出与地面的真相进行比较时, 它们只在标准节点中使用。但是, 在序列到顺序的网络中, 标签本身是在训练过程中输入到网络的一部分, 因为它们被输入到解码器中。

为了利用这些输入变量, 我们将通过计算节点传递它们。我们首先将 `input_sequence` to `raw_input` 设置为方便步骤。然后, 我们对 `label_sequence` 进行多次修改, 以便它能够与我们的网络一起使用。现在你只需要相信我们以后会好好利用这些东西。

首先, 我们将 `label_sequence` 中的第一个元素切掉, 这样它就缺少了句子起始标记。这是因为在训练和评估过程中, 解码器总是首先使用该令牌。当地面的真相没有被灌输到解码器中时, 我们仍然会在一个句子启动令牌中输入, 所以我们要一致地查看解码器作为一个以实际值开始的序列。

然后, 通过从序列 `raw_labels` 中获取 `first` 元素获取 `label_sequence_start` 。这将用于组成一个序列, 它是解码器的第一个输入, 无论我们是训练还是解码。最后, 最后两个语句设置了一个实际的序列, 具有正确的动态轴, 以送入解码器。函数 `sequence.scatter` 接受 `label_sentence_start` (即 `<s>` ) 的内容, 并将其转换为包含序列起始符号的第一个元素和包含0的元素的其余部分的序列。

_unchecked_

### 让我们创建 LSTM 循环

In [14]:
def LSTM_layer(input, 
               output_dim, 
               recurrence_hook_h=C.sequence.past_value, 
               recurrence_hook_c=C.sequence.past_value):
    # we first create placeholders for the hidden state and cell state which we don't have yet
    dh = C.placeholder(shape=(output_dim), dynamic_axes=input.dynamic_axes)
    dc = C.placeholder(shape=(output_dim), dynamic_axes=input.dynamic_axes)

    # we now create an LSTM_cell function and call it with the input and placeholders
    LSTM_cell = C.layers.LSTM(output_dim)
    f_x_h_c = LSTM_cell(dh, dc, input)
    h_c = f_x_h_c.outputs

    # we setup the recurrence by specifying the type of recurrence (by default it's `past_value` -- the previous value)
    h = recurrence_hook_h(h_c[0])
    c = recurrence_hook_c(h_c[1])

    replacements = { dh: h.output, dc: c.output }
    f_x_h_c.replace_placeholders(replacements)

    h = f_x_h_c.outputs[0]
    c = f_x_h_c.outputs[1]

    # and finally we return the hidden state and cell state as functions (by using `combine`)
    return C.combine([h]), C.combine([c])

_unchecked_

### 练习 1: 创建编码器

我们将使用我们刚才定义的 LSTM 复发。请记住, 它的功能签名是:

`def LSTM_layer(input, output_dim, recurrence_hook_h=sequence.past_value, recurrence_hook_c=sequence.past_value):`

它返回一个元组 `(hidden_state, hidden_cell)` 。我们将完成下面的四练习。如果可能, 在看答案之前先试用一下。

1. 创建编码器 (将 `output_dim` 和 `cell_dim` 设置为 `hidden_dim` 我们先前定义的)。

2. 将 `num_layers` 设置为高于1的东西, 并创建一叠 LSTMs 来表示编码器。

3. 获取编码器的输出, 并把它放到正确的形式传递到解码器 [硬]

4. 反转 `input_sequence` 的顺序 (这已被显示为特别有助于机器翻译)

In [15]:
# 1.
# Create the encoder (set the output_dim to hidden_dim which we defined earlier).

(encoder_output_h, encoder_output_c) = LSTM_layer(input_sequence, hidden_dim)

# 2.
# Set num_layers to something higher than 1 and create a stack of LSTMs to represent the encoder.
num_layers = 2
output_h = C.alias(input_sequence) # get a copy of the input_sequence
for i in range(0, num_layers):
    (output_h, output_c) = LSTM_layer(output_h.output, hidden_dim)

# 3.
# Get the output of the encoder and put it into the right form to be passed into the decoder [hard]
thought_vector_h = C.sequence.first(output_h)
thought_vector_c = C.sequence.first(output_c)

thought_vector_broadcast_h = C.sequence.broadcast_as(thought_vector_h, label_sequence)
thought_vector_broadcast_c = C.sequence.broadcast_as(thought_vector_c, label_sequence)

# 4.
# Reverse the order of the input_sequence (this has been shown to help especially in machine translation)
(encoder_output_h, encoder_output_c) = LSTM_layer(input_sequence, 
                                                  hidden_dim, 
                                                  C.sequence.future_value, 
                                                  C.sequence.future_value)

_unchecked_

### 练习 2: 创建解码器

在我们的序列到序列网络的基本版本中, 解码器通过将解码器的初始状态设置为编码器的最终隐藏状态来生成给定输入序列的输出序列。隐藏状态由一个元组 `(encoder_h, encoder_c)` , 其中 `h` 表示输出隐藏状态, `c` 表示 LSTM 单元格的值。

除了设置解码器的初始状态外, 我们还需要给解码器 LSTM 一些输入。第一个元素将始终是特殊的序列开始标记 `<s>` 。之后, 我们有两种方法来连接解码器的输入: 一个在训练期间, 另一个在评估期间 (即在经过训练的网络上生成序列)。

对于训练, 解码器的输入是来自训练数据的输出序列, 也称为输入序列的标签。在评估过程中, 我们会将输出从网络重定向回解码器作为它的历史。让我们先设置培训的输入..。

In [16]:
decoder_input = C.element_select(is_first_label, label_sentence_start_scattered, C.sequence.past_value(label_sequence))

_unchecked_

在上面, 我们使用函数 `element_select` , 它将返回给定条件的两个选项之一 `is_first_label` 。请记住, 我们正在使用序列, 所以当解码器 LSTM 运行它的输入将与网络一起展开。以上允许我们有一个动态的输入, 将返回一个特定的元素, 考虑到我们目前正在处理的时间步长。

因此, `decoder_input` 将是 `label_sentence_start_scattered` (这只是 `<s>` ), 当我们在第一时间的步骤, 否则它将返回 `past_value` (即前面的元素, 我们目前在什么时间步骤) 的 `label_sequence` 。

接下来, 我们需要设置实际的解码器。以前, 对于编码器, 我们执行了以下操作:

In [17]:
(output_h, output_c) = LSTM_layer(input_sequence, hidden_dim,
                                  recurrence_hook_h=C.sequence.past_value, 
                                  recurrence_hook_c=C.sequence.past_value)

_unchecked_

为了能够将解码器的第一个隐藏状态设置为与编码器的最终隐藏状态相等, 我们可以利用参数 `recurrence_hookH` and `recurrent_hookC` 。默认值 `past_value` 是一个返回时间序列中的元素的函数 `t-1` 。看看你能不能弄清楚怎么设置

1. 为解码器 LSTM 创建定期挂钩。

2. 提示: 您必须创建一个 `lambda operand:` , 您将使用我们前面使用的 `is_first_label` 掩码, 以及编码器输出的 `thought_vector_broadcast_h` 和 `thought_vector_broadcast_c` 表示。

3. 使用定期挂钩, 创建解码器。

4. 提示: 我们将再次使用 `LSTMP_component_with_self_stabilization()` 函数并再次使用 `hidden_dim` `output_dim` 和 `cell_dim` 。

5. 创建一个具有多个层的解码器。请注意, 您必须使用不同的递归钩子, 以便将反馈到图层堆栈中的底层。

In [18]:
# 1.
# Create the recurrence hooks for the decoder LSTM.

recurrence_hook_h = lambda operand: C.element_select(is_first_label, 
                                                     thought_vector_broadcast_h, 
                                                     C.sequence.past_value(operand))
recurrence_hook_c = lambda operand: C.element_select(is_first_label, 
                                                   thought_vector_broadcast_c, 
                                                   C.sequence.past_value(operand))

# 2.
# With your recurrence hooks, create the decoder.

(decoder_output_h, decoder_output_c) = LSTM_layer(decoder_input, hidden_dim, recurrence_hook_h, recurrence_hook_c)

# 3.
# Create a decoder with multiple layers.
# Note that you will have to use different recurrence hooks for the lower layers

num_layers = 3
decoder_output_h = C.alias(decoder_input)
for i in range(0, num_layers):
    if (i > 0):
        recurrence_hook_h = C.sequence.past_value
        recurrence_hook_c = C.sequence.past_value
    else:
        recurrence_hook_h = lambda operand: C.element_select(
            is_first_label, thought_vector_broadcast_h, C.sequence.past_value(operand))
        recurrence_hook_c = lambda operand: C.element_select(
            is_first_label, thought_vector_broadcast_c, C.sequence.past_value(operand))

    (decoder_output_h, decoder_output_c) = LSTM_layer(decoder_output_h.output, hidden_dim,
                                                      recurrence_hook_h, recurrence_hook_c)

_unchecked_

### 练习 3: 完全连接的层 (网络输出)

现在, 我们几乎已经到了定义网络的最后。我们所需要做的就是拿出解码器的输出, 然后通过一个线性层来运行它。最终, 它将被放入一个 `softmax` , 以获得可能的输出词的概率分布。但是, 我们将把它包括在我们的标准节点 (下面) 的一部分。

1. 添加线性层 (权重矩阵、偏置参数、次数和加号) 以获得网络的最终输出

In [19]:
# 1.
# Add the linear layer

W = C.parameter(shape=(decoder_output_h.shape[0], label_vocab_dim), init=C.glorot_uniform())
B = C.parameter(shape=(label_vocab_dim), init=0)
z = C.plus(B, C.times(decoder_output_h, W))

_unchecked_

## 将模型放在一起

与以上我们已经定义了一些网络, 并要求您定义它的一部分作为练习。这里让我们把整个事情放到一个叫做 `create_model()` 的函数中。请记住, 所有这些操作都是创建网络的骨架, 该框架定义数据将如何流经它。尚未通过它运行数据。

In [20]:
def create_model():

    # Source and target inputs to the model
    input_seq_axis = C.Axis('inputAxis')
    label_seq_axis = C.Axis('labelAxis')

    raw_input = C.sequence.input_variable(
        shape=(input_vocab_dim), sequence_axis=input_seq_axis, name='raw_input')

    raw_labels = C.sequence.input_variable(
        shape=(label_vocab_dim), sequence_axis=label_seq_axis, name='raw_labels')

    # Instantiate the sequence to sequence translation model
    input_sequence = raw_input

    # Drop the sentence start token from the label, for decoder training
    label_sequence = C.sequence.slice(raw_labels, 1, 0,
                                    name='label_sequence') # <s> A B C </s> --> A B C </s>
    label_sentence_start = C.sequence.first(raw_labels)      # <s>

    # Setup primer for decoder
    is_first_label = C.sequence.is_first(label_sequence)  # 1 0 0 0 ...
    label_sentence_start_scattered = C.sequence.scatter(
        label_sentence_start, is_first_label)

    # Encoder
    stabilize = C.layers.Stabilizer()
    encoder_output_h = stabilize(input_sequence)
    for i in range(0, num_layers):
        (encoder_output_h, encoder_output_c) = LSTM_layer(
            encoder_output_h.output, hidden_dim, C.sequence.future_value, C.sequence.future_value)

    # Prepare encoder output to be used in decoder
    thought_vector_h = C.sequence.first(encoder_output_h)
    thought_vector_c = C.sequence.first(encoder_output_c)

    thought_vector_broadcast_h = C.sequence.broadcast_as(
        thought_vector_h, label_sequence)
    thought_vector_broadcast_c = C.sequence.broadcast_as(
        thought_vector_c, label_sequence)

    # Decoder
    decoder_history_hook = C.alias(label_sequence, name='decoder_history_hook') # copy label_sequence

    decoder_input = C.element_select(is_first_label, label_sentence_start_scattered, C.sequence.past_value(
        decoder_history_hook))

    decoder_output_h = stabilize(decoder_input)
    for i in range(0, num_layers):
        if (i > 0):
            recurrence_hook_h = C.sequence.past_value
            recurrence_hook_c = C.sequence.past_value
        else:
            recurrence_hook_h = lambda operand: C.element_select(
                is_first_label, thought_vector_broadcast_h, C.sequence.past_value(operand))
            recurrence_hook_c = lambda operand: C.element_select(
                is_first_label, thought_vector_broadcast_c, C.sequence.past_value(operand))

        (decoder_output_h, decoder_output_c) = LSTM_layer(
            decoder_output_h.output, hidden_dim, recurrence_hook_h, recurrence_hook_c)

    # Linear output layer
    W = C.parameter(shape=(decoder_output_h.shape[0], label_vocab_dim), init=C.glorot_uniform())
    B = C.parameter(shape=(label_vocab_dim), init=0)
    z = C.plus(B, C.times(stabilize(decoder_output_h), W))

    return z

_unchecked_

## 培训

现在我们已经创建了模型, 我们准备好训练网络并学习它的参数。对于序列-序列网络, 我们使用的损失是交叉熵。请注意, 我们必须从模型中找到 `label_sequences` 节点, 因为它是在我们的网络中定义的, 我们希望将模型的预测与该节点的输出进行比较。

In [21]:
model = create_model()
label_sequence = model.find_by_name('label_sequence')

# Criterion nodes
ce = C.cross_entropy_with_softmax(model, label_sequence)
errs = C.classification_error(model, label_sequence)

# let's show the required arguments for this model
print([x.name for x in model.arguments])

['raw_labels', 'raw_input']


_unchecked_

接下来, 我们将设置一组参数来驱动我们的学习, 我们将创建一个学习者, 并最终创建我们的培训师:

In [22]:
# training parameters
lr_per_sample = C.learning_parameter_schedule_per_sample(0.007)
minibatch_size = 72
momentum_schedule = C.momentum_schedule(0.9366416204111472, minibatch_size=minibatch_size)
clipping_threshold_per_sample = 2.3
gradient_clipping_with_truncation = True
learner = C.momentum_sgd(model.parameters,
                         lr_per_sample, momentum_schedule,
                         gradient_clipping_threshold_per_sample=clipping_threshold_per_sample,
                         gradient_clipping_with_truncation=gradient_clipping_with_truncation)
trainer = C.Trainer(model, (ce, errs), learner)

_unchecked_

现在, 我们将功能和标签从 `train_reader` 绑定到我们在网络定义中设置的输入。首先, 我们将定义一个方便函数, 在将读者的特征指向模型的参数时帮助查找参数名称。

In [23]:
# helper function to find variables by name
def find_arg_by_name(name, expression):
    vars = [i for i in expression.arguments if i.name == name]
    assert len(vars) == 1
    return vars[0]

train_bind = {
        find_arg_by_name('raw_input' , model) : train_reader.streams.features,
        find_arg_by_name('raw_labels', model) : train_reader.streams.labels
    }

_unchecked_

最后, 我们定义我们的培训回路, 并开始培训网络!

In [24]:
training_progress_output_freq = 100
max_num_minibatch = 100 if isFast else 1000

for i in range(max_num_minibatch):
    # get next minibatch of training data
    mb_train = train_reader.next_minibatch(minibatch_size, input_map=train_bind)
    trainer.train_minibatch(mb_train)

    # collect epoch-wide stats
    if i % training_progress_output_freq == 0:
        print("Minibatch: {0}, Train Loss: {1:.3f}, Train Evaluation Criterion: {2:2.3f}".format(i,
                        trainer.previous_minibatch_loss_average, trainer.previous_minibatch_evaluation_average))

Minibatch: 0, Train Loss: 4.234, Train Evaluation Criterion: 0.982


_unchecked_

## 模型评估: 贪婪解码

一旦我们有了一个训练有素的模型, 我们当然会想利用它来生成输出序列!在这种情况下, 我们将使用贪婪解码。这意味着我们将通过我们训练有素的网络运行一个输入序列, 当我们生成输出序列时, 我们将通过获取网络输出的 `hardmax()` 来一次完成一个元素。这在总体上显然不是最佳的。考虑到上下文, 某些单词可能在第一步是最有可能的, 但是在以后的输出中可能会优先考虑另一个单词。解码最优序列一般是难以解决的。但我们可以做的更好做的光束搜索, 我们保持在一些小的假设, 在每一步。然而, 贪婪的解码在序列到序列的网络中可以很好地工作, 因为在 RNN 中有那么多的上下文被保存着。

要做贪婪解码, 我们需要钩入我们的网络的以前的输出作为输入到解码器。在训练期间, 我们通过了 `label_sequences` (地面真相)。您将注意到在我们的 `create_model()` 函数上面有以下几行:

In [25]:
decoder_history_hook = C.alias(label_sequence, name='decoder_history_hook') # copy label_sequence
decoder_input = C.element_select(is_first_label, label_sentence_start_scattered, 
                                 C.sequence.past_value(decoder_history_hook))

_unchecked_

这为我们提供了一种在对其他东西进行训练后修改 `decoder_history_hook` 的方法。我们已经训练了我们的网络, 但是现在我们需要一种方法来评估它而不使用地面的真相。我们可以这样做:

In [26]:
model = create_model()

# get some references to the new model
label_sequence = model.find_by_name('label_sequence')
decoder_history_hook = model.find_by_name('decoder_history_hook')

# and now replace the output of decoder_history_hook with the hardmax output of the network
def clone_and_hook():
    # network output for decoder history
    net_output = C.hardmax(model)

    # make a clone of the graph where the ground truth is replaced by the network output
    return model.clone(C.CloneMethod.share, {decoder_history_hook.output : net_output.output})

# get a new model that uses the past network output as input to the decoder
new_model = clone_and_hook()

_unchecked_

`new_model`现在包含与它共享参数的原始网络的版本, 但对解码器具有不同的输入。即, 而不是将地面的真相标签放入解码器, 它将在网络产生的历史中喂养!

最后, 让我们看看它的样子, 如果我们训练, 并通过运行一个字的分析 ("a B a D") 通过我们的网络, 不断评估网络的输出每一个 `100` 迭代。通过这种方式, 我们可以直观地看到进步学习的最佳模式..。首先, 我们将定义一个更完整的 `train()` 操作。它基本上是相同的, 但有一些额外的培训参数包括;一些额外的智慧, 打印出统计, 我们沿着;我们现在看到的进展, 我们的数据作为世纪 (一个世纪是一个完整的通过培训数据);我们为上面描述的单个验证序列设置了一个读取器, 这样我们就可以直观地看到我们的网络在该序列中所取得的进展。

In [27]:
########################
# train action         #
########################

def train(train_reader, valid_reader, vocab, i2w, model, max_epochs):

    # do some hooks that we won't need in the future
    label_sequence = model.find_by_name('label_sequence')
    decoder_history_hook = model.find_by_name('decoder_history_hook')

    # Criterion nodes
    ce = C.cross_entropy_with_softmax(model, label_sequence)
    errs = C.classification_error(model, label_sequence)

    def clone_and_hook():
        # network output for decoder history
        net_output = C.hardmax(model)

        # make a clone of the graph where the ground truth is replaced by the network output
        return model.clone(C.CloneMethod.share, {decoder_history_hook.output : net_output.output})

    # get a new model that uses the past network output as input to the decoder
    new_model = clone_and_hook()

    # Instantiate the trainer object to drive the model training
    lr_per_sample = C.learning_rate_schedule(0.007, C.UnitType.sample)
    minibatch_size = 72
    momentum_time_constant = C.momentum_as_time_constant_schedule(1100)
    clipping_threshold_per_sample = 2.3
    gradient_clipping_with_truncation = True
    learner = C.momentum_sgd(model.parameters,
                             lr_per_sample, momentum_time_constant,
                             gradient_clipping_threshold_per_sample=clipping_threshold_per_sample,
                             gradient_clipping_with_truncation=gradient_clipping_with_truncation)
    trainer = C.Trainer(model, (ce, errs), learner)

    # Get minibatches of sequences to train with and perform model training
    i = 0
    mbs = 0

    # Set epoch size to a larger number for lower training error
    epoch_size = 5000 if isFast else 908241

    training_progress_output_freq = 100

    # bind inputs to data from readers
    train_bind = {
        find_arg_by_name('raw_input' , model) : train_reader.streams.features,
        find_arg_by_name('raw_labels', model) : train_reader.streams.labels
    }
    valid_bind = {
        find_arg_by_name('raw_input' , new_model) : valid_reader.streams.features,
        find_arg_by_name('raw_labels', new_model) : valid_reader.streams.labels
    }

    for epoch in range(max_epochs):
        loss_numer = 0
        metric_numer = 0
        denom = 0

        while i < (epoch+1) * epoch_size:
            # get next minibatch of training data
            mb_train = train_reader.next_minibatch(minibatch_size, input_map=train_bind)
            trainer.train_minibatch(mb_train)

            # collect epoch-wide stats
            samples = trainer.previous_minibatch_sample_count
            loss_numer += trainer.previous_minibatch_loss_average * samples
            metric_numer += trainer.previous_minibatch_evaluation_average * samples
            denom += samples

            # every N MBs evaluate on a test sequence to visually show how we're doing; also print training stats
            if mbs % training_progress_output_freq == 0:

                print("Minibatch: {0}, Train Loss: {1:2.3f}, Train Evaluation Criterion: {2:2.3f}".format(mbs,
                      trainer.previous_minibatch_loss_average, trainer.previous_minibatch_evaluation_average))

                mb_valid = valid_reader.next_minibatch(minibatch_size, input_map=valid_bind)
                e = new_model.eval(mb_valid)
                print_sequences(e, i2w)

            i += mb_train[find_arg_by_name('raw_labels', model)].num_samples
            mbs += 1

        print("--- EPOCH %d DONE: loss = %f, errs = %f ---" % (epoch, loss_numer/denom, 100.0*(metric_numer/denom)))
        return 100.0*(metric_numer/denom)

_unchecked_

现在我们已经定义了我们的三重要函数-- `create_model()` 和 `train()` , 让我们来使用它们:

In [28]:
# Given a vocab and tensor, print the output
def print_sequences(sequences, i2w):
    for s in sequences:
        print([i2w[np.argmax(w)] for w in s], sep=" ")

# hook up data
train_reader = create_reader(train_file, True)
valid_reader = create_reader(valid_file, False)
vocab, i2w = get_vocab(vocab_file)

# create model
model = create_model()

# train
error = train(train_reader, valid_reader, vocab, i2w, model, max_epochs=1)

Minibatch: 0, Train Loss: 4.234, Train Evaluation Criterion: 0.982
['</s>', '~T', '~T', 'K', '~EH', '</s>']
['</s>', '~T', '~T', 'K', '~EH', '</s>']
['</s>', '~T', '~T', 'K', '~EH', '</s>']
['</s>', '~T', '~T', 'K', '~EH', '</s>']
['</s>', '~T', '~T', 'K', '~EH', '</s>']
['</s>', '~T', '~T', 'K', '~EH', '</s>']
['</s>', '~T', '~T', 'K', '~EH', '</s>']
['</s>', '~T', '~T', 'K', '~EH', '</s>']
['</s>', '~T', '~T', 'K', '~EH', '</s>']
['</s>', '~T', '~T', 'K', '~EH', '</s>']
--- EPOCH 0 DONE: loss = 3.801315, errs = 87.306223 ---


In [29]:
# Print the training error
print(error)

87.30622332060211


_unchecked_

## 任务

注意错误非常高。这主要是由于我们迄今所做的最低限度的培训。请将 `epoch_size` 更改为更高的数字并重新运行 `train` 函数。这可能需要相当长的时间, 但您将看到错误的明显减少。

_unchecked_

## 后续步骤

序列到序列模型的一个重要扩展, 特别是在处理长序列时, 是使用注意机制。注意的背后的想法是允许解码器, 首先, 看看任何隐藏的状态输出从编码器 (而不是只使用最后的隐藏状态), 其次, 了解多少注意支付给每个隐藏的状态给出的上下文。这使输出的单词在每次步骤时 `t` 不仅依赖于最终的隐藏状态和它前面的单词, 而是取决于*所有*输入隐藏状态的加权组合!

在本教程的下一版本中, 我们将讨论如何在序列网络中包含注意事项。